<img src="data/ccdproc_banner.svg" width=500 alt="ccdproc logo">

# ccdproc

+ Code: https://github.com/astropy/ccdproc
+ Documentation: https://ccdproc.readthedocs.io/
+ Detailed image reduction guide using ccdproc: https://github.com/astropy/ccd-reduction-and-photometry-guide
+ Report bugs/issues here: https://github.com/astropy/ccdproc/issues
    - Most of the bugs reported at or after the January AAS workshop are fixed in ccdproc 2.2 😃

## ccdproc can

+ Perform essential image reduction operations
+ Combine images
+ Tools for working on sets of images

## In this section we will

+ learn how ccdproc represents images in Python
+ learn how to reduce images

<hr>

## Preliminaries

In [ ]:
# If you work with files regularly then you should use 
# pathlib, which is part of the Python standard library
from pathlib import Path

# Initial imports -- note the import of CCDData from astropy 
from astropy.nddata import CCDData
from astropy import units as u
import ccdproc as ccdp

from matplotlib import pyplot as plt

# This displays static plots in the notebook...it will always work
%matplotlib inline

# Use this instead for interactive plots, but note that you may need 
# to install ipympl for it to work
# %matplotlib widget

# This function displays  images reasonably nicely with minimal effort
from convenience_functions import show_image

### ⚠️ If you get an error...

..the problem is likely that `ccdproc` is not installed. Install it with: `pip install ccdproc`

### Load an image

Although images are almost always stored as FITS files, you should use `CCDData.read()` instead of [`astropy.io.fits`](https://docs.astropy.org/en/stable/io/fits/index.html) to open single images to use in `ccdproc`. Ways to work with collections of images (like a directory of images) will be discussed later.

In [ ]:
raw_stars = CCDData.read('data/kelt-16-b-S001-R001-C084-r.fit.gz')

An image loaded this way has a few convient attributes:


In [ ]:
# Display the data 
print(raw_stars.data)

In [ ]:
# Display the header
print(raw_stars.header)

The header is a regular `astropy.io.fits` header object, so you can access keywords like it is a Python dictionary. For example, to get the exposure time by itself, run the cell below:

In [ ]:
raw_stars.header['exposure']

#### Displaying images

Images from a telescope are typically very different, in terms of the histogram of values, than other images. The function `show_image`, included along with this notebook, scales and stretches the data to bring out some of the detail that is not visible using matplotlib defaults.

In [ ]:
show_image(raw_stars)

### ⚠️ watch out for this

When you create a `CCDData` object you need to provide a unit. The images we are using today have a unit defined in the FITS header (in the `BUNIT` keyword). You can specify a unit, if necessary, as described here: https://ccdproc.readthedocs.io/en/latest/ccddata.html#getting-data-in

## Image reduction


### For a more detailed discussion see the [CCD reduction guide](https://mwcraig.github.io/ccd-as-book/01-00-Understanding-an-astronomical-CCD-image.html)

You can think of an astronomical image as being build from a few sources:

$$
\text{raw image} = \text{bias} + \text{noise} + \text{dark current} + \text{flat} \times (\text{sky} + \text{stars}).
$$

Solving for the counts just from the stars gives the following:

$$
\text{stars} + \text{noise} = \frac{\text{raw image} - \text{bias} - \text{dark current}}{\text{flat}} - \text{sky}
$$

**It is *impossible* to remove the noise from the raw image because the noise is
random.**

`ccdproc` provides functions for each of the operations on the right: bias subtraction, dark subtraction and flat correction. The output of one step becomes the input of the next step.


### About the calibration images

Each of the calibration images below was generate by combining 10 or 20 individual images. We will discuss image combination in the next notebook of this tutorial.

### Bias subtraction

In this and each of the next steps we will read in the calibration file and use the appropriate function from `ccdproc` to apply it to the data. 

Bias calibration is done by the function `ccdproc.subtract_bias`.

In [ ]:
# Read the data

bias = CCDData.read('data/combined_bias.fit.gz')

In [ ]:
stars_minus_bias = ccdp.subtract_bias(raw_stars, bias)

Let's check how the bias subtraction affected the image. In particular, we are looking to see whether the vertical bright line in the initial image has been removed.

In [ ]:
show_image(stars_minus_bias)

### Dark subtraction

The amount of dark current in an image depends on the exposure time of the image. The combined dark needs to be properly scaled if its exposure time doesn't match the exposure time of the science image. The function `ccdproc.subtract_dark` can handle that scaling for you as long as indicate which FITS keyword contains the exposure time and what the exposure time units are, or explicitly provide both the exposure time of the dark frame and the exposure time of the bias-subtracted science image.

In [ ]:
# Read the dark frame

dark = CCDData.read('data/combined_dark_90.000.fit.gz')

Note that the first argument to `ccdproc.subtract_dark` is the bias-subtracted image.

In [ ]:
stars_minus_bias_minus_dark = ccdp.subtract_dark(stars_minus_bias, dark,
                                                 exposure_time='exposure',
                                                 exposure_unit=u.second)

Dark calibration should have removed the "hot" pixels in the initial image. Let's check that it did.

In [ ]:
show_image(stars_minus_bias_minus_dark)

### Flat correction

The final step in the calibration is to apply flat corrections using `ccdproc.flat_correct`. 

In [ ]:
flat = CCDData.read('data/combined_flat_filter_r.fit.gz')

As with the other reduction steps, the first argument is the output of the previous reduction step and the second is the calibration image to be applied.

In [ ]:
stars_calibrated = ccdp.flat_correct(stars_minus_bias_minus_dark, flat)

This calibrated image will not look that much different than the one with bias and dark subtraction, particular because this is a small section of a much larger (4k × 4k) image so there is not much variation in sensitivity across this sub-image.

In [ ]:
show_image(stars_calibrated)

## Why not just subtract and/or divide arrays?

### ccdproc handles uncertainty propagation -- and more!

Both the bias and dark calibration images used in this tutorial have uncertainty arrays stored with them. By using `ccdproc` instead of, for example, simply subtracting the bias values from the raw science image values, those uncertainties are properly propagated. That continues with each of the subsequent steps.

Another advantage of using `ccdproc` is that things like scaling of the dark calibration exposure time to that of the science image (and proper scaling of any uncertainty in the dark frame) are also taken care of.

Finally, there is a built in mechanism for automatically updating headers as you procede.

## A more compact approach

The function `ccd_process`, described [briefly here](https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#basic-processing-with-a-single-command) and in [much more detail here](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.ccd_process.html#ccdproc.ccd_process), performs all of the reductions steps on a single image with one call.

### Exercise

Read the descriptions of `ccd_process` and try to use it to calibrate the image `raw_stars` that you loaded earlier in this notebook.

Note that for the images in this tutorial you do not need provide values for `trim` or `oscan` .

If you prefer to load the solution, uncomment the line below and run the cell.

*We will use this solution below.*

In [ ]:
# %load ccd_process_solution.py

## Processing a directory of images

Though there a number of ways using plain Python that can be used to process a directory with several images, `ccdproc` provides the `ImageFileCollection` to make it easier to work with those files.

### Create a temporary directory of images for this tutorial

To keep the download size for the tutorial reasonable, only one science image was included in the data. The cell below creates a directory with a few copies of the science image for use in the next step.

In [ ]:
# You need to run this but you do not need to understand it in any 
# detail for the rest of the tutorial.

sample_images = Path('data-copy')
sample_images.mkdir(exist_ok=True)

n_copies = 5

for i in range(5):
    this_image_name = 'kelt-16-copy-{}.fit'.format(i)
    raw_stars.write(str(sample_images / this_image_name))

### `ImageFileCollection`

The main thing that an [`ImageFileCollection`](https://ccdproc.readthedocs.io/en/latest/image_management.html) provides is a way to iterate over the FITS files in a directory. You can filter by FITS keyword values and iterate over just the headers, just the data, or the entire `CCDData` object.

First, let's create the collection.

In [ ]:
# The directory data-copy was created in the previous code cell

ifc = ccdp.ImageFileCollection('data-copy')

Now let's reduce all of the images in that directory. 

The code below does a few things:

1. Create a folder, `data-reduced`, for the reduced images.
1. Iterate over each of the images in the folder `data-copy`. 
1. Use `ccd_process` to calibrate each image.
2. Save the reduced images to the folder.

In [ ]:
reduced_path = Path('data-reduced')
reduced_path.mkdir()

for ccd, name in ifc.ccds(imagetyp='light', # Only include images of type light
                          filter='r', # and with filter r
                          return_fname=True  # Also provide the name of the file
                   ):
    
    ccd_reduced = ccdp.ccd_process(ccd,
                                   master_bias=bias,
                                   dark_frame=dark,
                                   exposure_key='exposure', exposure_unit=u.second,
                                   dark_scale=True,
                                   master_flat=flat)
    ccd_reduced.write(reduced_path / name)

### More about `ImageFileCollection`

An [`ImageFileCollection`](https://ccdproc.readthedocs.io/en/latest/image_management.html) also provides straightforward access to a table with the FITS keywords from the image headers. Access the table via the `summary` attribute. In the cell below we disply the file name, exposure time, image type, object and filter for each image.

In [ ]:
ifc.summary['file', 'imagetyp', 'exposure', 'object', 'filter']

*Note:* If you display the full table then the `comment` and `history` entries are often huge.

Astropy tables provide [powerful ways of grouping entries and iterating over those groups](https://docs.astropy.org/en/stable/table/operations.html). That can be very helpful for processing images grouped by object or by filter.

By default Astropy tables only print the first ten and last ten rows to keep the size of the table reasonable. If you want to display the whole table, use the [Astropy table pretty print method](https://docs.astropy.org/en/stable/table/access_table.html#pprint-method), either `ifc.summary.pprint(max_lines=-1)` or `ifc.summary.pprint_all()`.